## Nonpolar solvation free energy with an implicit solvent model

This notebook is an implementation of the implicit solvent model described in "A Simple Electrostatic Model for the Hard-Sphere Solute Component of Nonpolar Solvation" (Christopher D. Cooper and Jaydeep P. Bardhan) for the nonpolar component of solvation free energy ($\Delta G_{np}$). This model decomposes $\Delta G_{np}$ in the free energy spent in the cavity creation ($\Delta G_{cav}$), plus the energy of the solute-solvent dispersion interaction ($\Delta G_{disp}$), so that $\Delta G_{np} = \Delta G_{cav} + \Delta G_{disp}$.

We use the well known test case from David Mobley et al "Small Molecule Hydration Free Energies in Explicit Solvent: An Extensive Test of Fixed-Charge Atomistic Simulations" to test our model. We expect you to download the `prmtop` files from its [Supporting Information](https://pubs.acs.org/doi/10.1021/ct800409d), and point to that folder in the `mobley_test_folder` variable. Also, you'll need to download [`msms`](https://mgl.scripps.edu/people/sanner/html/msms_home.html) to generate meshes. Please point to the `msms` binary in the `msms_bin` variable.

In [ ]:
from pathlib import Path

mobley_test_folder = str(Path.home())+'/Downloads/prmcrd/'
msms_bin = 'msms'

### A capacitor model for $\Delta G_{cav}$

The cavity energy model is based in a simple *capacitor* model, that models the solute as two concentric conducting surfaces, one on the dielectric interface (the solvent-excluded surface) and another one around the first solvation shell, one water radius out (on the solvent-accessible surface). Then, $\Delta G_{cav}$ is the electrostatic energy stored in the capacitor, such that it sustains a potential $\phi_{static}$ inside the dielectric interface and 0 outside the shell.

The electrostatic potential due to a surface charge $\sigma$ is

\begin{equation}
\phi(\mathbf{r}) = \oint_\Gamma \frac{\sigma(\mathbf{r}')}{4\pi\epsilon|\mathbf{r}-\mathbf{r}'|}d\mathbf{r}' = \frac{1}{\epsilon}V_\mathbf{r}(\sigma)
\end{equation}

Then, the surface charge densities on the SES and SAS ($\sigma_{SES}$ and $\sigma_{SAS}$) required to have potential $\phi_{static}$ on the SES and 0 on the SAS is solution to

\begin{equation}
\left[
\begin{matrix}
V_{diel} & V_{diel} \\
V_{shell} & V_{shell} 
\end{matrix}
\right]
\left(
\begin{matrix}
\sigma_{diel}\\
\sigma_{shell}
\end{matrix}
\right)
=
\epsilon_{shell}
\left(
\begin{matrix}
\phi_{static}\\
0
\end{matrix}
\right)
\end{equation}

where $\epsilon_{shell}$ is the dielectric constant in the first hydration shell. Having the surface charges, the energy stored in the capacitor is

\begin{equation}
\Delta G_{cav} = \oint \phi_{static} \sigma_{shell}(\mathbf{r}) d\mathbf{r}
\end{equation}

Next, we will use the [`bempp`](https://bempp.com/) library to solve the matrix equation above numerically, and compute $\Delta G_{cav}$.

#### Function definitions

First, we import the required libraries and define a function to generate the grid in `bempp` format. This reads in positions and atomic radii and uses [`msms`](https://mgl.scripps.edu/people/sanner/html/msms_home.html) to generate mesh files (`.vert` and `.face`) that are then read by `bempp`.

In [ ]:
import numpy
import bempp.api
import os
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

def generate_grid(atom_pos, atom_radius, msms_bin, radius_increase=0, p=1.4, d=2):
    
    xyzr_data = numpy.zeros((N_atom, 4))
    xyzr_data[:,:3] = atom_pos[:,:]
    xyzr_data[:,-1] = atom_radius[:] + radius_increase

    # generate xyzr and call msms
    numpy.savetxt('atom_aux.xyzr', xyzr_data)
    cmd = msms_bin + ' -if atom_aux.xyzr -of mesh_aux -d %1.3f -p %1.3f -no_header'%(d, p)
    os.system(cmd)
   
    # read meshes into bempp
    face = open('mesh_aux.face','r').read()
    vert = open('mesh_aux.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)
    
    os.system('rm atom_aux.xyzr mesh_aux.*');

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

Next, we define a function to generate the right-hand side of the matrix equation, and another that will assemble the matrix and solve the linear system

In [ ]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static

def solve_matrix(grid_diel, grid_shell, phi_static, eps_s):

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_shell = bempp.api.function_space(grid_shell, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_shell = grid_shell.leaf_view.entity_count(0)

    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_shell.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_shell)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_shell, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_shell, space_shell)
    M22   = laplace.single_layer(space_shell, space_shell, space_shell) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print ('Not converged, %i iterations'%info)
    elif info<0:
        print ('Solver diverges')
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]
    
    return sigma_d, elements_d

Finally, we can generate a function to compute the energy

In [ ]:
def compute_energy_cav(sigma_diel, elements_diel, phi_static):
    
    qe = 1.60217662e-19
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    N_diel = len(sigma_diel)
    area_diel = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_diel[i] = elements_diel[i].geometry.volume

    conv_factor = (1000*eps_0)/(qe**2*Na*1e10*4.184)

    energy = 0.5*conv_factor*numpy.sum(area_diel*sigma_diel*phi_static)

    return energy

#### Running the model

##### Mesh generation

We'll now generate meshes with `msms` where `mesh_diel` and `mesh_shell` are the  mesh files of the SES and SAS, respectively. Use the `molecule_name` variable to point at the solute you want to simulate (as named in the Mobley test case folder).

In [ ]:
molecule_name = '1112_tetrachloroethane'

First, let's read the `prmtop` and `crd` files to obtain the Lennard-Jones parameters and coordinates. For this, we'll use the [`ParmEd`](https://parmed.github.io/ParmEd/html/index.html) library, available in [`pip`](https://pypi.org/project/ParmEd/) and [`conda`](https://anaconda.org/omnia/parmed).

In [ ]:
import parmed.amber

mol_param = parmed.amber.AmberParm(mobley_test_folder + molecule_name + '.prmtop')
N_atom = mol_param.ptr('NATOM')
atom_type = mol_param.parm_data['ATOM_TYPE_INDEX']
atom_radius = numpy.zeros(N_atom)
atom_depth = numpy.zeros(N_atom)

for i in range(N_atom):
    atom_radius[i] = mol_param.LJ_radius[atom_type[i]-1]
    atom_depth[i] = mol_param.LJ_depth[atom_type[i]-1]

mol_crd = numpy.loadtxt(mobley_test_folder + molecule_name + '.crd', skiprows=2)
mol_crd.flatten()
atom_pos = numpy.reshape(mol_crd, (N_atom, 3)) 

Now we can generate the mesh for the dielectric and shell surfaces

In [ ]:
grid_diel = generate_grid(atom_pos, atom_radius, msms_bin)
grid_shell = generate_grid(atom_pos, atom_radius, msms_bin, radius_increase = 1.4, p=0.01)

In [ ]:
phi_static = 10.7 #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

eps_s = 7.75

sigma_diel, elements_diel = solve_matrix(grid_diel, grid_shell, phi_static, eps_s)
dGcav = compute_energy_cav(sigma_diel, elements_diel, phi_static)
print('dG_cav = %1.5f kcal/mol'%dGcav)

### A continuum integral model for $\Delta G_{disp}$

The solute-solvent dispersion interaction can be computed by integrating the Lennard-Jones potential in the solvent

\begin{equation}
\Delta G_{disp} = \sum_i \int_{solv}\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^{12}} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}
\end{equation}

where the sum is over the solute molecules, $A_i$ and $B_i$ are the Lennard-Jones parameters for atom $i$, and $\rho_w$ = 0.0336 Angstrom$^{-3}$ is the solvent number density. Using the divergence theorem, we write this equation as

\begin{equation}
\Delta G_{disp} = \sum_i \oint_{shell} \rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r}  
\end{equation}

To account for the fact that the water number density increases near surfaces, we add an extra  1.4 Angstrom layer beyond the shell surface, and with an augmented density of 1.75$\rho_w$. Then, the energy can be written as

\begin{equation}
\Delta G_{disp} = \sum_i \int_{layer}1.75\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} + \int_{solv}\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}  
\end{equation}

where the first term is the integral over the new layer, and the second over the rest of the solvent. We can write the first integral as

\begin{equation}
\int_{layer}1.75\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} = \int_{layer+solv}1.75\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} - \int_{solv}1.75\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r}
\end{equation}

which can be written as surface integrals

\begin{equation}
\int_{layer}1.75\rho_w \left(\frac{A_i}{|\mathbf{r}-\mathbf{r}_i|^12} - \frac{B_i}{|\mathbf{r}-\mathbf{r}_i|^6}\right) d\mathbf{r} = \oint_{shell} 1.75\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} - \oint_{shell, out} 1.75\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r}. 
\end{equation}

Then, the dispersion energy is

\begin{equation}
\Delta G_{disp} = \sum_i \left( \oint_{shell} 1.75\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} -  \oint_{shell, out} 0.75\rho_w \frac{\partial}{\partial \mathbf{n}}\left( \frac{A_i}{90|\mathbf{r} - \mathbf{r}_i|^{10}} - \frac{B_i}{12|\mathbf{r} - \mathbf{r}_i|^4}\right) \text{d}\mathbf{r} \right)
\end{equation}

and the following function computes the surface integral above

In [ ]:
def disp_integral(atom_pos, atom_radius, atom_depth, grid):

    qe = 1.60217662e-19
    Na = 6.0221409e23
    rho_w = 0.0336# 1/angs3 number density of water at standard conditions
    water_radius = 1.7683 # angs
    water_depth = 0.1520 # kcal/mol
      
    N_atom = len(atom_radius)

    N_panel = grid.leaf_view.entity_count(0)
    vertices = grid.leaf_view.vertices 
    triangles = grid.leaf_view.elements
    elements = list(grid.leaf_view.entity_iterator(0))

    area = numpy.zeros(N_panel)
    center = numpy.zeros((N_panel,3))
    normal = numpy.zeros((N_panel,3))
    for i in range(N_panel):
        area[i] = elements[i].geometry.volume
        center[i,:] = numpy.average(elements[i].geometry.corners[:],axis=1)
        v1 = elements[i].geometry.corners[:,1] - elements[i].geometry.corners[:,0]
        v2 = elements[i].geometry.corners[:,2] - elements[i].geometry.corners[:,0]
        normal[i,:] = numpy.cross(v1,v2)/(2*area[i]) 

    integral_i = numpy.zeros(N_atom)
    for i in range(N_atom):
        r_local = center - atom_pos[i,:]
        r_norm = numpy.sqrt(numpy.sum(r_local**2, axis=1))
        rdn = numpy.sum(r_local*normal, axis=1)
        epsilon = numpy.sqrt(water_depth*atom_depth[i])
    
        A = epsilon*(water_radius + atom_radius[i])**12
        B = 2*epsilon*(water_radius + atom_radius[i])**6
          
        integral_i[i] = numpy.sum((A/(9*r_norm**12) - B/(3*r_norm**6)) * rdn * area)   

    integral = rho_w*numpy.sum(integral_i) #kcal/mol
    return integral

First, we need to generate the mesh for the outer interface, 2.8 Angstroms away from the original solvent-excluded surface, between the new layer and the solvent.

In [ ]:
grid_shell_out = generate_grid(atom_pos, atom_radius, msms_bin, radius_increase = 2.8, p=0.01)

Next, we call this function to integrate over both surfaces

In [ ]:
integral_shell = disp_integral(atom_pos, atom_radius, atom_depth, grid_shell)
integral_shell_out = disp_integral(atom_pos, atom_radius, atom_depth, grid_shell_out)

and finally compute the energy considering the increase in density of the outer shell

In [ ]:
rho_increased = 1.75
dGdisp = integral_shell*rho_increased - integral_shell_out*(rho_increased-1)

print('dG_disp = %1.5f kcal/mol'%dGdisp)